In [ ]:
# hide
%load_ext nb_black

<IPython.core.display.Javascript object>

In [ ]:
# default_exp servers

<IPython.core.display.Javascript object>

## FastAPIUvicornServer

In [ ]:
import time
import subprocess

from pathlib import Path

from will_it_saturate.servers import BaseServer
from will_it_saturate.registry import register_model

<IPython.core.display.Javascript object>

In [ ]:
# export


@register_model
class FastAPIUvicornServer(BaseServer):
    name: str = "fastAPI/uvicorn"

    def get_pid(self):
        kwargs = {"shell": True, "capture_output": True, "text": True}
        output = subprocess.run(
            f"ps aux | grep will_it_saturate.fastapi.main:app", **kwargs
        )
        lines = [l for l in output.stdout.split("\n") if len(l) > 0 and "grep" not in l]
        if len(lines) > 0:
            pid = lines[0].split()[1]
            return pid

    @property
    def started(self):
        return self.get_pid() is not None

    def start_server(self):
        subprocess.Popen(
            [
                "uvicorn",
                "--workers",
                "4",
                "--host",
                str(self.host),
                "--port",
                str(self.port),
                "--no-access-log",
                "will_it_saturate.fastapi.main:app",
            ]
        )
        # subprocess.Popen(["uvicorn", "will_it_saturate.fastapi.main:app"])

    def stop_server(self):
        subprocess.check_output(["kill", self.get_pid()])
        time.sleep(1)  # dunno why this is necessary

    def start(self):
        if not self.started:
            self.start_server()

    def stop(self):
        if self.started:
            self.stop_server()

<IPython.core.display.Javascript object>

### Usage

In [ ]:
# dont_test

# server = FastAPIUvicornServer()
# server.start()
# print(server.get_pid())
# server.stop()
# print("stopped..")
# print(server.get_pid())
# print(server.started)

<IPython.core.display.Javascript object>

In [ ]:
server = FastAPIUvicornServer()
print(server.get_pid())

None


<IPython.core.display.Javascript object>

In [ ]:
server.start()

<IPython.core.display.Javascript object>

Usage: uvicorn [OPTIONS] APP
Try 'uvicorn --help' for help.

Error: No such option: -w


In [ ]:
server.stop()

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [25235]


<IPython.core.display.Javascript object>

## DjangoGunicornWSGIServer

In [ ]:
# export


@register_model
class DjangoGunicornWSGIServer(BaseServer):
    name: str = "django/gunicorn/wsgi"

    def get_pids(self):
        kwargs = {"shell": True, "capture_output": True, "text": True}
        output = subprocess.run(f"ps aux | grep will_it_saturate.django.wsgi", **kwargs)
        lines = [l for l in output.stdout.split("\n") if len(l) > 0 and "grep" not in l]
        print(len(lines))
        pids = []
        for line in lines:
            pid = line.split()[1]
            pids.append(pid)
        return pids

    @property
    def started(self):
        return len(self.get_pids()) > 0

    def start_server(self):
        subprocess.Popen(
            [
                "gunicorn",
                "--backlog",
                "10000",
                "-w",
                "8",
                "-b" f":{self.port}",
                "will_it_saturate.django.wsgi",
            ]
        )
        time.sleep(2)

    def stop_server(self):
        kill_command = ["kill"]
        kill_command.extend(self.get_pids())
        subprocess.check_output(kill_command)
        time.sleep(1)  # dunno why this is necessary

    def start(self):
        if not self.started:
            self.start_server()

    def stop(self):
        if self.started:
            self.stop_server()

<IPython.core.display.Javascript object>

In [ ]:
# dont_test

server = DjangoGunicornWSGIServer()
server.start_server()
print(server.get_pids())
server.stop()
print(server.get_pids())
print(server.started)

9
['25638', '25637', '25636', '25635', '25630', '25634', '25633', '25632', '25631']
9
9
0
[]
0
False


<IPython.core.display.Javascript object>

# Caddy

In [ ]:
# export


@register_model
class CaddyFileServer(BaseServer):
    name: str = "caddy"

    def get_pids(self):
        kwargs = {"shell": True, "capture_output": True, "text": True}
        output = subprocess.run(f"ps aux | grep -i caddy", **kwargs)
        lines = [l for l in output.stdout.split("\n") if len(l) > 0 and "grep" not in l]
        print(len(lines))
        pids = []
        for line in lines:
            pid = line.split()[1]
            pids.append(pid)
        return pids

    @property
    def started(self):
        return len(self.get_pids()) > 0

    def start_server(self):
        subprocess.Popen(
            [
                "caddy",
                "file-server",
                "--listen",
                f"0.0.0.0:{self.port}",
            ]
        )
        time.sleep(2)

    def stop_server(self):
        kill_command = ["kill"]
        kill_command.extend(self.get_pids())
        subprocess.check_output(kill_command)
        time.sleep(1)  # dunno why this is necessary

    def start(self):
        if not self.started:
            self.start_server()

    def stop(self):
        if self.started:
            self.stop_server()

<IPython.core.display.Javascript object>

In [ ]:
# dont_test

server = CaddyFileServer(port=5001)
server.start_server()
print(server.get_pids())
server.stop()
print(server.get_pids())
print(server.started)

{"level":"warn","ts":1648802602.167109,"logger":"admin","msg":"admin endpoint disabled"}
{"level":"info","ts":1648802602.1672041,"logger":"tls.cache.maintenance","msg":"started background certificate maintenance","cache":"0x140008c3dc0"}
{"level":"info","ts":1648802602.167552,"logger":"tls","msg":"cleaning storage unit","description":"FileStorage:/Users/jochen/Library/Application Support/Caddy"}
{"level":"info","ts":1648802602.167604,"logger":"tls","msg":"finished cleaning storage units"}
{"level":"info","ts":1648802602.16818,"msg":"autosaved config (load with --resume flag)","file":"/Users/jochen/Library/Application Support/Caddy/autosave.json"}
2022/04/01 10:43:22 Caddy 2 serving static files on 0.0.0.0:5001


1
['11319']
1
1


{"level":"info","ts":1648802604.331993,"msg":"shutting down apps, then terminating","signal":"SIGTERM"}
{"level":"warn","ts":1648802604.332008,"msg":"exiting; byeee!! 👋","signal":"SIGTERM"}
{"level":"info","ts":1648802604.33333,"logger":"tls.cache.maintenance","msg":"stopped background certificate maintenance","cache":"0x140008c3dc0"}
{"level":"info","ts":1648802604.333366,"msg":"shutdown complete","signal":"SIGTERM","exit_code":0}


0
[]
0
False


# Nginx

In [ ]:
# export


@register_model
class NginxFileServer(BaseServer):
    name: str = "nginx/minimal"

    def get_pids(self):
        kwargs = {"shell": True, "capture_output": True, "text": True}
        output = subprocess.run(f"ps aux | grep -i nginx", **kwargs)
        lines = [l for l in output.stdout.split("\n") if len(l) > 0 and "grep" not in l]
        print(len(lines))
        pids = []
        for line in lines:
            pid = line.split()[1]
            pids.append(pid)
        return pids

    @property
    def started(self):
        return len(self.get_pids()) > 0

    def create_config(self):
        cwd = Path.cwd()
        server_root = str(cwd)
        config = f"""
daemon off;
worker_processes 4;
error_log /dev/null;
pid /tmp/nginx.pid;
events {{}}
http {{
    server {{
        listen {self.port};
        sendfile           off;
        tcp_nopush         off;
        tcp_nodelay        off;
        root {server_root};
    }}
}}
        """
        config_path = cwd / "nginx.conf"
        with config_path.open("w") as f:
            f.write(config)
        return config_path

    def start_server(self):
        config_path = self.create_config()
        subprocess.Popen(
            [
                "nginx",
                "-c",
                str(config_path),
            ]
        )
        time.sleep(2)

    def stop_server(self):
        kill_command = ["kill"]
        kill_command.extend(self.get_pids())
        subprocess.check_output(kill_command)
        time.sleep(1)  # dunno why this is necessary

    def start(self):
        if not self.started:
            self.start_server()

    def stop(self):
        if self.started:
            self.stop_server()


@register_model
class NginxSendfileServer(NginxFileServer):
    name: str = "nginx/sendfile"

    def create_config(self):
        cwd = Path.cwd()
        server_root = str(cwd)
        config = f"""
error_log /dev/null;
pid /tmp/nginx.pid;
worker_processes 4;
daemon off;
events {{}}
http {{
    server {{
        listen    {self.port};
        sendfile           on;
        sendfile_max_chunk 1m;
        tcp_nopush         on;
        tcp_nodelay        on;
        keepalive_timeout  65;
        root {server_root};
    }}
}}
        """
        print(config)
        config_path = cwd / "nginx.conf"
        with config_path.open("w") as f:
            f.write(config)
        return config_path

<IPython.core.display.Javascript object>

In [ ]:
# dont_test

# server = NginxFileServer(port=5001)
server = NginxSendfileServer(port=5001)
server.start_server()
print(server.get_pids())
server.stop()
print(server.get_pids())
print(server.started)


daemon off;
events {}
http {
    server {
        listen 5001;
        sendfile        on;
        sendfile_max_chunk 1m;
        root /Users/jochen/projects/will_it_saturate;
    }
}
        
2
['39178', '39177']
2
2
0
[]
0
False


<IPython.core.display.Javascript object>

In [ ]:
# hide
# dont_test

from nbdev.export import notebook2script

notebook2script()

Converted 00_index.ipynb.
Converted 01_config.ipynb.
Converted 01_host.ipynb.
Converted 02_file.ipynb.
Converted 03_registry.ipynb.
Converted 04_epochs.ipynb.
Converted 10_servers.ipynb.
Converted 11_fastapi_main.ipynb.
Converted 12_django_views.ipynb.
Converted 13_django_handlers.ipynb.
Converted 14_django_http.ipynb.
Converted 15_django_asgi.ipynb.
Converted 15_django_async_settings.ipynb.
Converted 15_django_settings.ipynb.
Converted 15_django_urls.ipynb.
Converted 15_django_wsgi.ipynb.
Converted 16_minio.ipynb.
Converted 16_servers_started_locally.ipynb.
Converted 16_servers_started_via_docker.ipynb.
Converted 20_clients.ipynb.
Converted 21_benchmark_client_implementations.ipynb.
Converted 22_gevent_client.ipynb.
Converted 30_control_server.ipynb.
Converted 31_control_client.ipynb.
Converted 32_control_cli.ipynb.
Converted 40_results.ipynb.
Converted 41_repositories.ipynb.
Converted 42_sqlite_repository.ipynb.
Converted 50_benchmark_without_benchmark.ipynb.
Converted 51_benchmark_r

<IPython.core.display.Javascript object>